In [ ]:
#importing the required packages
import numpy as np
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib as mlt

In [ ]:
#Fetching the training and test sets
test = pd.read_csv('titanic-test.csv')
train = pd.read_csv('titanic-train.csv')

In [ ]:
#Inspecting the dimensions of out training and test sets
print('Dimension of train: {0}'.format(train.shape))
print("Dimension of test data: {0}".format(test.shape))

In [ ]:
train.info()

In [ ]:
train.head()

In [ ]:
#Check survival Rates for Each of the Classes grouped by Sex
sex_pivot = train.pivot_table(index="Sex",values="Survived")
sex_pivot.plot.bar()
plt.show()

In [ ]:
#Plot the Survival rate by P'class alone
class_pivot = train.pivot_table(index = 'Pclass',values = 'Survived',aggfunc = np.mean)
class_pivot.plot.bar()
plt.show()

In [ ]:
#Exploring and Converting the Age Column
#train['Age'].describe() # Average age is 30 years,
under_30 = train.query('Age <=30')
under_30.shape[0]/train.shape[0] 
# 46% of the Passengers are 30 years and below. (irrespective of is.na) Nan will be treated l8r

In [ ]:
#Lets see how many people survived sef..
survived = train.query('Survived ==1')
died = train.query('Survived ==0')
print(survived.shape); 
print(died.shape)

In [ ]:
#Ploting the Histogram Distribution
survived.Age.plot.hist(alpha = 0.5, color = 'red', bins =50) # Lucky folks
died.Age.plot.hist(alpha = 0.5, color = 'blue', bins =50) # May their sould rest in peace
plt.legend(['Survived','Died'])
plt.show()

In [ ]:
len(survived)/(len(survived) + len(died))

More People Survivie between 20 and 30 years than any other age range and more people within this age range didnt survive when compared to other age groups

In [ ]:
#Lets see their death-sity. Attimes histogram can be confusing at time....
survived.Age.plot.density(color = 'yellow')
died.Age.plot.density(color = 'magenta')
plt.legend(['Survived','Died'])
plt.show() # Their deathsity's distributon is similar.

In [ ]:
cut_points_1 = [-1, 0, 5, 12, 18, 25, 60, 100] # Randomly Selected, by my church mind
Agelabels = ['Missing', 'Infant', 'Child', 'Teenager', 'Young Adult','Adult','Senior'] #Categories

#function the processes  Age an
def ProcessAge(df, cut_points,label_names):
    
    # -0.5 for missing ages
    df['Age'] = df['Age'].fillna(-0.5) 
    
    df['Age_Categories'] = pd.cut(df['Age'],cut_points,labels = label_names)
    return df

In [ ]:
train = ProcessAge(train, cut_points = cut_points_1, label_names= Agelabels)
test = ProcessAge(test,cut_points= cut_points_1, label_names= Agelabels)
train.query("Age_Categories == 'Infant'")

train.pivot_table('Survived', index = 'Age_Categories', aggfunc= np.mean).plot.bar()
plt.show() 

#Interestingly! The graph below shows that more infants survived that any other age_categories

In [ ]:
#Lets split the PClass column into dummy columns to cater for 
#1st, 2nd and 3rd Clas seperately

def create_dummies(df, column_name):
    dummies = pd.get_dummies(df[column], prefix = column_name)
    df = pd.concat([df,dummies],axis =1)
    return df

In [ ]:
for column in ['Pclass','Sex','Age_Categories']:
    train = create_dummies(train,column)
    test = create_dummies(test,column)

In [ ]:
train.columns

In [ ]:

from sklearn.linear_model import LogisticRegression

#Creating a Logistic Regression object
titanic_model = LogisticRegression()

#Features we'll be using to train our model
features = ['Pclass_1', 'Pclass_2', 'Pclass_3', 'Sex_female', 'Sex_male',
            'Age_Categories_Missing','Age_Categories_Infant','Age_Categories_Child',
            'Age_Categories_Teenager','Age_Categories_Young Adult', 
            'Age_Categories_Adult','Age_Categories_Senior']

'''
NB
Some Features are more important than the other. (Thats for another day)
As per the Sklearn documentation, some classifiers are built in with an 
attribute which returns feature importances by simply typing in .featureimportances.

You can try this out..
'''

In [ ]:
train[features].head()

In [ ]:
from sklearn.model_selection import train_test_split

all_x = train[features]
all_y = train['Survived']

#Split the train test into train and test sets
train_X, test_X, train_y, test_y = train_test_split(all_x, all_y,
                                                    test_size = 0.3, random_state = 0)


In [ ]:
#Oya lets Train Our Model
titanic_model.fit(train_X, train_y)

In [ ]:
#generating predictions
predictions = titanic_model.predict(test_X)

In [ ]:
# We'll be using confusion matrix to evaluate predictions from our model..

from sklearn.metrics import confusion_matrix

titanic_confuse_matrix = confusion_matrix(test_y,predictions)
titanic_confuse_matrix

In [ ]:
#Visualizing the confusion matrix using seaborn package
import seaborn as sns
sns.heatmap(titanic_confuse_matrix, annot=True)

In [ ]:
#Time to measure the accuracy of the model
from sklearn.metrics import accuracy_score

accuracy  = accuracy_score(test_y,predictions)
print(accuracy) #78.7 not bad... Can your model predict a better accuracy score.. Give it a try

In [ ]:
#Till next time, Peace Out